In [ ]:
#detach和data的一些问题

In [13]:
import torch
from torch.nn import init
from torch.autograd import Variable
t1 = torch.FloatTensor([1., 2.])
v1 = Variable(t1)
t2 = torch.FloatTensor([2., 3.])
v2 = Variable(t2)
v3 = v1 + v2
print("v3:",v3)
v3_detached = v3.detach()
v3_dt = v3_detached.add_(t1)          #detach 后对V3_detach变化，结果。。。。。
print("v3data:",v3_dt,"v3:",v3)
v3_detached.data.add_(t1) # 修改了 v3_detached Variable中 tensor 的值
print(v3, v3_detached)    # v3 中tensor 的值也会改变

v3: tensor([3., 5.])
v3data: tensor([4., 7.]) v3: tensor([4., 7.])
v4: [ 8. 14.]
tensor([5., 9.]) tensor([5., 9.])


#CNN 网络

In [1]:
import torch
import torch.nn as nn
import torchvision 
import torchvision.transforms as transforms

In [2]:
# Device configuration
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [14]:
# Hyper parameters
num_epochs = 5
num_class = 10
batch_size = 100
lr= 0.002

In [15]:
# MNIST dataset
train_dataset = torchvision.datasets.MNIST(
    root='../../data/', train=True, transform=transforms.ToTensor(), download='True')
test_dataset = torchvision.datasets.MNIST(
    root='../../data/', train=False, transform=transforms.ToTensor())

# Data loader
train_loader = torch.utils.data.DataLoader(dataset=train_dataset,
                                          batch_size=batch_size,
                                          shuffle=True)
test_loader = torch.utils.data.DataLoader(dataset=test_dataset,
                                         batch_size=batch_size,
                                         shuffle=False)

In [22]:
class cnn(nn.Module):
    def __init__(self,num_class):
        super(cnn,self).__init__()
        self.conv1 = nn.Sequential(
            nn.Conv2d(in_channels=1,
                     out_channels=16,
                     kernel_size=5,
                     stride = 1,
                     padding = 2),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2),
        )
        self.conv2 = nn.Sequential(
            nn.Conv2d(16,32,5,1,2),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2),
        )
        self.out = nn.Linear(32*7*7,num_class)
        
    def forward(self,x):
        x= self.conv1(x)
        x = self.conv2(x)
        x =x.view(x.size(0),-1)
        output = self.out(x)
        return output
        

In [23]:
cnnmn = cnn(10).to(device)
print(cnnmn)

cnn(
  (conv1): Sequential(
    (0): Conv2d(1, 16, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
    (1): ReLU()
    (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (conv2): Sequential(
    (0): Conv2d(16, 32, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
    (1): ReLU()
    (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (out): Linear(in_features=1568, out_features=10, bias=True)
)


In [24]:
optimizer = torch.optim.Adam(cnnmn.parameters(),lr=learning_rate)
loss_F = nn.CrossEntropyLoss()

In [26]:
total_step = len(train_loader)
for epoch in range(num_epochs):
    for i, (images,labels) in enumerate(train_loader):
        images = images.to(device)
        labels = labels.to(device)
        
        outputs = cnnmn(images)
        loss = loss_F(outputs,labels)
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        if (i+1)%100 ==0:
            print("epoch:[{}/{}],step: {}/{} ,loss:{:.5f}".format(epoch,num_epochs,i+1,total_step,loss.item()))

epoch:[0/5],step: 100/600 ,loss:0.12153
epoch:[0/5],step: 200/600 ,loss:0.13469
epoch:[0/5],step: 300/600 ,loss:0.21621
epoch:[0/5],step: 400/600 ,loss:0.04917
epoch:[0/5],step: 500/600 ,loss:0.04166
epoch:[0/5],step: 600/600 ,loss:0.10131
epoch:[1/5],step: 100/600 ,loss:0.11748
epoch:[1/5],step: 200/600 ,loss:0.06102
epoch:[1/5],step: 300/600 ,loss:0.04045
epoch:[1/5],step: 400/600 ,loss:0.07078
epoch:[1/5],step: 500/600 ,loss:0.02743
epoch:[1/5],step: 600/600 ,loss:0.07096
epoch:[2/5],step: 100/600 ,loss:0.01856
epoch:[2/5],step: 200/600 ,loss:0.06146
epoch:[2/5],step: 300/600 ,loss:0.01493
epoch:[2/5],step: 400/600 ,loss:0.08378
epoch:[2/5],step: 500/600 ,loss:0.11389
epoch:[2/5],step: 600/600 ,loss:0.05562
epoch:[3/5],step: 100/600 ,loss:0.01138
epoch:[3/5],step: 200/600 ,loss:0.08703
epoch:[3/5],step: 300/600 ,loss:0.07008
epoch:[3/5],step: 400/600 ,loss:0.01786
epoch:[3/5],step: 500/600 ,loss:0.00826
epoch:[3/5],step: 600/600 ,loss:0.01657
epoch:[4/5],step: 100/600 ,loss:0.04648


In [27]:
# Test the model
cnnmn.eval()  # eval mode(batch norm uses moving mean/variance 
              #instead of mini-batch mean/variance)
with torch.no_grad():
    correct = 0
    total = 0
    for images, labels in test_loader:
        images = images.to(device)
        labels = labels.to(device)
        outputs = cnnmn(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
        
    print('Test accuracy of the model on the 10000 test images: {} %'
          .format(100 * correct / total))

Test accuracy of the model on the 10000 test images: 98.6 %


In [28]:
torch.save(cnnmn,'./cnnmnist')

F:\ProgramData\Anaconda3\lib\site-packages\torch\serialization.py:251: UserWarning: Couldn't retrieve source code for container of type cnn. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "


In [42]:
torch.save(cnnmn.state_dict(),'cnnmn_state_dict')

In [43]:
cnnminst = torch.load('cnnmnist')

In [48]:
print(cnnminst)
cnnminst.state_dict()

cnn(
  (conv1): Sequential(
    (0): Conv2d(1, 16, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
    (1): ReLU()
    (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (conv2): Sequential(
    (0): Conv2d(16, 32, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
    (1): ReLU()
    (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (out): Linear(in_features=1568, out_features=10, bias=True)
)


OrderedDict([('conv1.0.weight',
              tensor([[[[ 0.0023, -0.1526, -0.0023,  0.2144,  0.0836],
                        [-0.2964, -0.2174,  0.0938, -0.0502,  0.0677],
                        [-0.1625, -0.2886,  0.1582,  0.0388,  0.3091],
                        [-0.3041, -0.0343,  0.1562,  0.2064,  0.0312],
                        [-0.2355,  0.1499, -0.0874,  0.2762,  0.2690]]],
              
              
                      [[[-0.1480,  0.2575, -0.0645,  0.0025, -0.3564],
                        [-0.1300,  0.1611,  0.3622, -0.1073, -0.4075],
                        [-0.2277, -0.1003,  0.2977,  0.2519,  0.1003],
                        [-0.1242, -0.1106,  0.0376,  0.1994, -0.0861],
                        [-0.2071, -0.2286,  0.1410,  0.2784, -0.0612]]],
              
              
                      [[[-0.1050, -0.0447, -0.0937, -0.3985, -0.3922],
                        [ 0.1110,  0.2718,  0.2591, -0.1160,  0.0785],
                        [-0.0238,  0.0688,  0.0712, 

In [45]:
cnnm = cnnminst.load_state_dict(torch.load('cnnmn_state_dict'))

In [47]:
print(cnnm.weight)

AttributeError: 'NoneType' object has no attribute 'weight'